# NOTEBOOK DEPLOYEMENT CLOUD D'UNE APPLICATION DE MISE EN OEUVRE DES CLUSTERS DE CALCUL PYSPARK( Jobs Spark)

# 1 Introduction;

L'objet de ce projet est de deployer une application Spark jobs dans le cloud sur une plateforme Amazon Web Service.
La conception de ce projet necessite de suivre la démarche analytique suivante:

# a_Etablir une  surveillance du temps de calcul sur l'environnement local en vue d'une comparaison avec les performances en temps de l'environnement cloud.

# b_Urbaniser la localisation  des données d'entrée et de sortie sur les differents disques
    

# c_Charger les données
      Explorez Spark Dataframe au format 'image'
      Nous garderons l'origine pour obtenir le chemin et extraire l'étiquette
      Autre façon de traiter l'image
    
# d_Utiliser un CNN comme extracteur de fonctionnalités

###   ===>   Faire une caractérisation  des images :
        Définir la logique de chargement et de caractéristiques des images dans une UDF Pandas
    
###  ===> Procéder à une réduction dimensionnelle
        Transformer des tableaux en vecteurs pour effectuer une réduction
        Initialiser et appliquer PCA
        Transformation inverse : des vecteurs au tableau - c'est-à-dire la lisibilité des pandas
            
# c_Collecter et Sauvegarder les résultats en format parquet en vue d'une exploitation ultérieure.
          


# 2 Importation des librairies

Dans ce cahier, nous gérons via pySpark, sql & ml, la caractérisation et la réduction de dimension d'une collection d'images.
L'objectif est de construire une architecture évolutive pour effectuer les transformations attendues et permettre une croissance rapide de la collection d'images en gardant un temps de calcul viable.

In [1]:
# packages standard
import pandas as pd
import numpy as np
import time
import os
import io
############################
import sys
import pyspark
import findspark
from platform import python_version
############################
#Packlages Pyspark
############################
from PIL import Image,ImageDraw
from typing import Iterator
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SparkSession,Row
from PIL import Image, ImageDraw
from pyspark.sql.functions import col, pandas_udf, PandasUDFType, element_at, split,udf
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, ArrayType, BinaryType
from pyspark.ml.image import ImageSchema
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import PCA
import pyarrow
#transform
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.types import *
##################################
#Packages Keras
##################################
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array,load_img
from tensorflow.keras import Model
import tensorflow as tf


In [4]:
#Selection de l'environnement de deploiement
Environnement_Test_Local=False
    
#===================================================================================================================    
if Environnement_Test_Local:        
    findspark.init()
    # 4 Configuration des variables utilisateurs sur l'environnement de deploiement de Test après chargement de Python,Java et Spark dans le serveur windows 

    os.environ['PYSPARK_PYTHON'] = sys.executable
    os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable          

    ## Urbanisation en local

    PATH='C:\\Users\\dgama\\Desktop\\OPENCLASSROOMS_INFO\\REPERTOIRE_P8_BIS'

    PATH_R='C:\\Users\\dgama\\Desktop\\OPENCLASSROOMS_INFO\\REPERTOIRE_P8_TEMP'

    PATH_Data = PATH+'\Test2'
    PATH_Result = PATH_R+'\Results'
    print("********Bienvenue sur votre environnement de Test local pour jobs spark***********"+\
         "\n\nLa version Python extrait de l environnement Conda pyspark_env est:"\
         ,sys.version +"\nLa version de l'application Spark est:"\
         ,pyspark.__version__ +"\nL'OS de l'environnement de deploiement est:"\
         ,sys.platform +"\n\nPATH:        "+\
         PATH+"\nPATH_Data:   "+\
         PATH_Data +"\nPATH_Result:"+ PATH_Result)
#==========================================================================================================================       
else:
   
    # 4 Configuration des variables utilisateurs sur l'environnement de deploiement de Test après chargement de Python,Java et Spark dans le serveur windows 

    ## Urbanisation en local

    PATH='s3://myprojet8bucket'

    PATH_DATA='s3://myprojet8bucket/Input-Test/*'

    PATH_FEATURES='s3://myprojet8bucket/Output_Data/features/'

    PATH_FINAL_FEATURES='s3://myprojet8bucket/Output_Data/final_features/'

    PATH ='s3://myprojet8bucket'
    PATH_Data = PATH+'/Input-Test/*'
    PATH_FEATURES = PATH+'/Output/Features'
    PATH_FINAL_FEATURES=PATH+'/Output/Final_Features'
    print("********Bienvenue sur votre environnement EMR Notebook _Amazon Web Service pour jobs spark***********"+\
         "\n\nLa version Python extrait de l environnement Aws_Pyspark est:"\
         ,sys.version +"\nLa version de l'application Spark est:"\
         ,pyspark.__version__ +"\nL'OS de l'environnement de deploiement est:"\
         ,sys.platform +"\n\nPATH:        "+\
          PATH+"\nPATH_DATA:   "+\
          PATH_Data+"\nPATH_FEATURES: "\
          +PATH_FEATURES +"\nPATH_FINAL_FEATURES: "\
           +PATH_FINAL_FEATURES)

#================================

********Bienvenue sur votre environnement de production Amazon Web Service pour jobs spark***********

La version Python extrait de l environnement Aws_Pyspark est: 3.10.9 | packaged by conda-forge | (main, Feb  2 2023, 20:14:58) [MSC v.1929 64 bit (AMD64)]
La version de l'application Spark est: 3.3.2
L'OS de l'environnement de deploiement est: win32

PATH:        s3://myprojet8bucket
PATH_DATA:   s3://myprojet8bucket/Input-Test/*
PATH_FEATURES: s3://myprojet8bucket/Output/Features
PATH_FINAL_FEATURES: s3://myprojet8bucket/Output/Final_Features


In [ ]:
# 7 Session Spark

spark = (SparkSession
             .builder
             .appName('P8')
             .master('local')
             .config("spark.sql.parquet.writeLegacyFormat", 'true')
             .getOrCreate()
        )



In [ ]:
# check wether arrow should be enabled by this setting
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")


In [ ]:
sc = spark.sparkContext

In [ ]:
# open spark UI for app monitoring
spark

In [ ]:
# function to load data into into a spark_d
df_img = spark.read.format("binaryFile") \
  .option("pathGlobFilter", "*.jpg") \
  .option("recursiveFileLookup", "true") \
  .load(PATH_Data)

In [ ]:

# number of images in this sample
df_img.count()

In [ ]:
df_img.show(5)

In [ ]:
### Path de l'image conservé et nous ajoutons une colonne contenant les labels de chaque image

df_img = df_img.withColumn('label', element_at(split(df_img['path'], '/'),-2))
print(df_img.printSchema())
print(df_img.select('path','label').show(5,False))


In [ ]:
# 8 Préparation du Modèle

model = MobileNetV2(weights='imagenet',
                    include_top=True,
                    input_shape=(224,224,3))

new_model = Model(inputs=model.input,
                  outputs=model.layers[-2].output)



In [ ]:
new_model.summary()

In [ ]:
# get weights as broadcasted variable over nodes (provide a copy to each node)

brodcast_weights = sc.broadcast(new_model.get_weights())

In [ ]:
# 9 Caractérisation (Featurizer)  et Reduction dimensionnelle(Reducer)
'''
Nous utilisons le transfer learning pour featuriser les images .
Nous allons procéder à la réduction des la dimension des torseurs d'images en transformons les matrices d'images
en Vecteurs d'images'''

start = time.perf_counter()
def model_fn():
    """
    Returns a MobileNetV2 model with top layer removed 
    and broadcasted pretrained weights.
    """
    model = MobileNetV2(weights='imagenet',
                        include_top=True,
                        input_shape=(224, 224, 3))
    for layer in model.layers:
        layer.trainable = False
    new_model = Model(inputs=model.input,
                  outputs=model.layers[-2].output)
    new_model.set_weights(brodcast_weights.value)
    return new_model

In [ ]:
def preprocess(content):
    """
    Preprocesses raw image bytes for prediction.
    """
    img = Image.open(io.BytesIO(content)).resize([224, 224])
    arr = img_to_array(img)
    return preprocess_input(arr)

def featurize_series(model, content_series):
    """
    Featurize a pd.Series of raw images using the input model.
    :return: a pd.Series of image features
    """
    input = np.stack(content_series.map(preprocess))
    preds = model.predict(input)
    # For some layers, output features will be multi-dimensional tensors.
    # We flatten the feature tensors to vectors for easier storage in Spark DataFrames.
    output = [p.flatten() for p in preds]
    return pd.Series(output)

@pandas_udf('array<float>', PandasUDFType.SCALAR_ITER)
def featurize_udf(content_series_iter):
    '''
    This method is a Scalar Iterator pandas UDF wrapping our featurization function.
    The decorator specifies that this returns a Spark DataFrame column of type ArrayType(FloatType).

    :param content_series_iter: This argument is an iterator over batches of data, where each batch
                              is a pandas Series of image data.
    '''
    # With Scalar Iterator pandas UDFs, we can load the model once and then re-use it
    # for multiple data batches.  This amortizes the overhead of loading big models.
    model = model_fn()
    for content_series in content_series_iter:
        yield featurize_series(model, content_series)


In [ ]:
featurized_df = df_img.repartition(20).select(col("path"),
                                            col("label"),
                                            featurize_udf("content").alias("features")
                                           )

featurized_df.show()

In [ ]:
featurized_df.printSchema()

In [ ]:

featurized_df.write.mode("overwrite").parquet(PATH_Result)

featurized_dt = pd.read_parquet(PATH_Result, engine='pyarrow')

featurized_dt.head()

In [ ]:
# get the number of paritions

print(featurized_df.rdd.getNumPartitions())

### On valide que les vecteur des caracterisque des images sint  bien de dimension 1280
featurized_dt.loc[0,'features'].shape

In [ ]:
# from Array to Vectors for PCA

array_to_vector_udf = udf(lambda l: Vectors.dense(l), VectorUDT())

vectorized_df = featurized_df.withColumn('cnn_vectors', array_to_vector_udf('features'))

vectorized_df.show(5, True)

In [ ]:
# 10 Initialisation et Application PCA

# Results may vary with larger dataset, creates an heavy task Action that would affect overall performance.

# reduce with PCA - set k Max to determine the adequate nb of principal components

pca = PCA(k=20, inputCol='cnn_vectors', outputCol='pca_vectors')
model = pca.fit(vectorized_df)

# apply pca reduction

reduced_df = model.transform(vectorized_df)

reduced_df.show(5, True)


In [ ]:
# 11 Transformation inverse: Des vecteurs vers un tableau - i.e. DataFrame Pandas

# from Array to Vectors for PCA
vector_to_array_udf = udf(lambda v: v.toArray().tolist(), ArrayType(FloatType()))

final_df = reduced_df.withColumn('features', vector_to_array_udf('pca_vectors'))

final_df.show(5, True)


In [ ]:
# 12 Collectes et Sauvegardes finales des Jobs Spark

# write local results on parquet file

final_df.write.mode('overwrite').parquet(PATH_Result)

# read local results from parquet file

pd_final_df = pd.read_parquet(PATH_Result, engine='pyarrow')

# size of the results df

pd_final_df.info(verbose=False, memory_usage="deep")

# overview
pd_final_df.head(5)

In [ ]:
# Fin du processus Job spark
spark.stop()